In [1]:
import os
import pandas as pd
import numpy as np

import json

#libraries for data visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
import seaborn as sns
sns.set_color_codes("pastel")

#map 
import folium
from folium import plugins

In [2]:
filepath = "/Users/reejungkim/Documents/Git/Production analysis/Online Retail.xlsx"
df = pd.read_excel(filepath, sheet_name = 'Online Retail')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [4]:
df.CustomerID = df.CustomerID.astype(str)

In [5]:
df['TotalPurchase'] = df.Quantity * df.UnitPrice

In [6]:
df['year-m'] = df['InvoiceDate'].dt.strftime('%Y-%m') #df['InvoiceDate'].apply(lambda x: x.strftime("%Y-%m"))
df.sort_values(by='InvoiceDate', ascending=True, inplace=True)

### load local json file

In [7]:
#json_file_path = r"./Users/reejungkim/Documents/geo_countries.json"
rfile = open("/Users/reejungkim/Documents/Git/geo_countries.json", "r", encoding='utf-8').read() 
jsonData = json.loads(rfile)

geo_json_data = pd.read_json("/Users/reejungkim/Documents/Git/geo_countries.json")
geo_json_data.head()

geo_json_data.features

geo_df = pd.json_normalize(geo_json_data.features)
geo_df.head()

### or load json file from git

In [8]:
import requests
url = 'https://raw.githubusercontent.com/reejungkim/User-Analysis/master/geo_countries.json'
jsonData = json.loads(requests.get(url).text)

# Map

In [14]:
geo_map = folium.Map(location=[37, -102], zoom_start=2) #, tiles='Stamen Toner') tiles='Stamen Terrain') tiles='cartodbpositron')

m = geo_map
m

## Add json data to the map

folium.GeoJson(jsonData, name='json_data').add_to(geo_map)
geo_map

## Reflect quantity ordered on the map

In [10]:
df_sales = df.groupby('Country').agg({'CustomerID': lambda customer: customer.count(),
                                        'Quantity': lambda quant: quant.sum(),
                                        'TotalPurchase': lambda price: price.sum()}).reset_index()
df_sales['AvgPurchasePerCustomer'] = df_sales.TotalPurchase/df_sales.CustomerID

In [15]:
folium.Choropleth(
    geo_data= jsonData,
    name= 'choropleth',
    data= df_sales,
    columns=['Country', 'AvgPurchasePerCustomer'],
    key_on='properties.sovereignt',
    fill_color= 'OrRd',
    fill_opacity=0.5,
    line_opacity=0.2,
    legend_name='Average puchase per customer',
    highlight=True,
    line_color='blue', 
    reset= True
).add_to(geo_map)

geo_map

In [16]:
choropleth = folium.Choropleth(
    geo_data= jsonData,
    name= 'choropleth',
    data= df_sales,
    columns=['Country', 'TotalPurchase', 'CustomerID'],
    key_on='properties.sovereignt',
    fill_color= 'YlGn', #'BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd',
    fill_opacity=0.5,
    line_opacity=0.2,
    legend_name='Total purchase',
    highlight=True,
    line_color='blue', 
    reset= True
).add_to(geo_map)

folium.LayerControl(collapsed=True).add_to(geo_map)
geo_map

## save as html file

In [17]:
geo_map.save('PurchaseByCountry.html')

In [ ]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')

fig = go.Figure(data=go.Choropleth(
    locations = df['CODE'],
    z = df['GDP (BILLIONS)'],
    text = df['COUNTRY'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '$',
    colorbar_title = 'GDP<br>Billions US$',
))

fig.update_layout(
    title_text='2014 Global GDP',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://www.cia.gov/library/publications/the-world-factbook/fields/2195.html">\
            CIA World Factbook</a>',
        showarrow = False
    )]
)

fig.show()

In [ ]:
import plotly.express as px

fig = px.choropleth(locations=["CA", "TX", "NY"], locationmode="USA-states", color=[1,2,3], scope="world")# scope='usa'
fig.show()

In [ ]:
for country in df.Country.unique():
    print(country)
    sns.distplot(df.Quantity.loc[df.Country==country],  color ='r', label=country)
    sns.distplot(df.TotalPurchase.loc[df.Country==country],  color ='b', label=country)
    plt.show()